# NLP for SMS spam filtering
Dataset used: UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
# import spark
from pyspark.sql import SparkSession

In [2]:
# create spark session
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
# read data
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [4]:
# rename columns
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



### Feature engineer a length column to take into account no of chars in text

In [6]:
# import length function
from pyspark.sql.functions import length

In [7]:
# add length column
data = data.withColumn('length', length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
# Show the difference between ham and spam
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### Feature Transformations

In [10]:
# import tokenizer
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [12]:
# create tokenizer
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

# create stop words remover
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')

# create count vectorizer
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')

# create idf
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

# create label indexer
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [13]:
# import vector assembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [14]:
# create vector assembler
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

### Model: We will be using Naive Bayes for this test

In [15]:
# import naive bayes
from pyspark.ml.classification import NaiveBayes

In [16]:
# create naive bayes model
nb = NaiveBayes()

### Create Pipeline

In [17]:
# import pipeline
from pyspark.ml import Pipeline

In [18]:
# create pipeline in order
data_prep_pipe = Pipeline(stages=[ham_spam_to_num, tokenizer, stopremove, count_vec, idf, clean_up])

In [30]:
# fit and transform the pipeline
cleaner = data_prep_pipe.fit(data)
cleaned = cleaner.transform(data)

In [31]:
# show label and features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



### Train test split

In [32]:
# split data
training, testing = cleaned.randomSplit([0.7, 0.3])

In [33]:
# fit model to training data
spam_predictor = nb.fit(training)

In [34]:
# transform model with test data
test_results = spam_predictor.transform(testing)


In [37]:
# show results
test_results.select(['label', 'prediction']).show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



### Model evaluation

In [38]:
# import evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
# create evaluator
acc_eval = MulticlassClassificationEvaluator()

In [41]:
# get accuracy
acc = acc_eval.evaluate(test_results)

# print accuracy
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9241187312861591
